In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import gzip
import zipfile
import shutil
from time import sleep
import pickle
from time import sleep
import sys
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [2]:
with open('query_dict/id_querry_spelled.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)
with open('title_dict/title_data.pickle', 'rb') as file:
    title_data = pickle.load(file)

In [3]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [4]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [5]:
model1 = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3')

In [6]:
neuro_feats = dict()
neuro_feats['use-multilingual-large/3_unnormed_titles'] = []

In [7]:
q_ids = all_groups['QueryId'].unique()
for q1 in tqdm(q_ids):
    corpus = []
    tmp = all_groups[all_groups.QueryId==q1]
    docs = tmp['DocumentId'].values
    for doc1 in docs:
        corpus.append(title_data[doc1])
    cur_q = id_querry_clean[q1]
    one_tok_txt = cur_q.split()
    Q_emb = model1(cur_q)
    for doc1 in corpus:
        D_emb = model1(doc1)
        neuro_feats['use-multilingual-large/3_unnormed_titles'].append(cosine_similarity(Q_emb.numpy(), D_emb.numpy()).flatten()[0])

100%|██████████| 6311/6311 [12:39:14<00:00,  7.22s/it]  


In [8]:
with open('use-multilingual-large_3_unnormed_titles.pickle', 'wb') as file:
    pickle.dump(neuro_feats, file)

In [2]:
model1 = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')

In [13]:
neuro_feats = dict()
neuro_feats['use-multilingual/3_unnormed_titles'] = []

In [14]:
q_ids = all_groups['QueryId'].unique()
for q1 in tqdm(q_ids):
    corpus = []
    tmp = all_groups[all_groups.QueryId==q1]
    docs = tmp['DocumentId'].values
    for doc1 in docs:
        corpus.append(title_data[doc1])
    cur_q = id_querry_clean[q1]
    one_tok_txt = cur_q.split()
    Q_emb = model1(cur_q)
    for doc1 in corpus:
        D_emb = model1(doc1)
        neuro_feats['use-multilingual/3_unnormed_titles'].append(cosine_similarity(Q_emb.numpy(), D_emb.numpy()).flatten()[0])

100%|██████████| 6311/6311 [1:21:52<00:00,  1.28it/s]


In [17]:
with open('use-multilingual_3_unnormed_titles.pickle', 'wb') as file:
    pickle.dump(neuro_feats, file)